In [1]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [3]:
ko_text = "뉴진스에는 매일 찾게 되고 언제 입어도 질리지 않는 진처럼 시대의 아이콘이 되겠다는 포부와 새로운 시대를 열겠다는 각오도 동시에 담겨 있다."
en_text = "NewJeans carries both the ambition of becoming an icon of the times, like jeans that you find yourself reaching for every day and never get tired of wearing, and the determination to open a new era."

# 모델과 토크나이저 불러오기
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 한국어를 영어로 번역
tokenizer.src_lang = "ko"
encoded_ko = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.get_lang_id("en"))
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True)
print(translated_text)

# 영어를 한국어로 번역
tokenizer.src_lang = "en"
encoded_en = tokenizer(en_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("ko"))
translated_text2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True)
print(translated_text2)

['New Jeans also contain the desire to be an icon of the age as they will be found every day and when they don’t wear, and to open a new age at the same time.']
['NewJeans는 시간의 아이콘이되기위한 야망, 당신이 매일 도달하고 착용에 지쳤을 때 결코 찾을 수있는 제인과 같은, 그리고 새로운 시대를 열기위한 결심을 가지고 있습니다.']


In [5]:
import gradio as gr
import torch

In [7]:
# CPU 디바이스로 설정 (DirectML 대신 CPU 사용)
device = torch.device("cpu")

# 번역 모델 초기화 및 CPU로 이동
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(device)

# 토크나이저 초기화
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 언어와 언어 코드 목록
languages = {
    "Korean": "ko",
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
    # 필요 시 더 추가
}

/root/miniconda3/envs/my_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# 번역 함수
def translate(text, source_lang, target_lang):
    tokenizer.src_lang = languages[source_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(device)  # CPU로 이동
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(languages[target_lang]))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

# Gradio 앱 빌드
with gr.Blocks() as app:
    gr.Markdown("# Multilingual Translation Service")

    with gr.Row():
        source_lang = gr.Dropdown(label="Source Language", choices=list(languages.keys()), value="Korean")
        target_lang = gr.Dropdown(label="Target Language", choices=list(languages.keys()), value="English")

    input_text = gr.Textbox(label="Input Text")
    output_text = gr.Textbox(label="Translated Text")
    translate_btn = gr.Button("Translate")

    # 번역 버튼 클릭 시 동작 설정
    translate_btn.click(fn=translate, inputs=[input_text, source_lang, target_lang], outputs=output_text)

# Gradio 앱 실행
app.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://bd7268f497ebe23d36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 읽기 기능 추가

In [13]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import soundfile as sf

In [14]:
# CPU 디바이스로 설정 (DirectML 대신 CPU 사용)
device = torch.device("cpu")

# 번역 모델과 토크나이저 초기화 및 CPU로 이동
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(device)
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 언어 목록 및 언어 코드
languages = {
    "Korean": "ko",
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
}

/root/miniconda3/envs/my_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
# Microsoft SpeechT5 TTS 모델 로드
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
speech_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)  # CPU로 이동
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)  # CPU로 이동

# 음성 특성을 위한 스피커 임베딩 데이터셋 로드
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

# 번역 및 TTS 함수
def translate_and_tts(text, source_lang, target_lang):
    # 언어 설정 후 번역
    tokenizer.src_lang = languages[source_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(device)  # 입력 데이터를 CPU로 이동
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(languages[target_lang]))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    
    # 대상 언어가 영어일 경우 TTS 수행
    tts_output_file = None
    if target_lang == "English":
        tts_output_file = convert_text_to_speech(translated_text)
    
    return translated_text, tts_output_file

# 텍스트를 음성으로 변환하는 함수
def convert_text_to_speech(text):
    inputs = processor(text=text, return_tensors="pt").to(device)
    
    # 스피커 임베딩 선택
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0).to(device)
    
    # TTS 모델과 vocoder를 사용하여 음성 생성
    speech = speech_model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
    
    # 생성된 음성을 파일로 저장
    temp_audio_file = "output_speech.wav"
    sf.write(temp_audio_file, speech.numpy(), samplerate=16000)
    
    return temp_audio_file

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [16]:
with gr.Blocks() as app:
    gr.Markdown("# Multilingual Translation Service with Microsoft SpeechT5 TTS")

    with gr.Row():
        source_lang = gr.Dropdown(label="Source Language", choices=list(languages.keys()), value="Korean")
        target_lang = gr.Dropdown(label="Target Language", choices=list(languages.keys()), value="English")

    input_text = gr.Textbox(label="Input Text")
    output_text = gr.Textbox(label="Translated Text")
    tts_audio = gr.Audio(label="Text-to-Speech Output")  # Audio player for speech output
    translate_btn = gr.Button("Translate and Speak")

    # Connect the button click event to the translation and TTS function
    translate_btn.click(fn=translate_and_tts, inputs=[input_text, source_lang, target_lang], outputs=[output_text, tts_audio])

app.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://5e80d6cfc33cab0106.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
